In [1]:
!git clone https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Super-Resolution

Cloning into 'a-PyTorch-Tutorial-to-Super-Resolution'...
remote: Enumerating objects: 47, done.
remote: Total 47 (delta 0), reused 0 (delta 0), pack-reused 47
Unpacking objects: 100% (47/47), done.


In [ ]:
!git clone https://github.com/Lornatang/CycleGAN-PyTorch

Cloning into 'CycleGAN-PyTorch'...
remote: Enumerating objects: 440, done.
remote: Counting objects: 100% (437/437), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 440 (delta 245), reused 426 (delta 238), pack-reused 3
Receiving objects: 100% (440/440), 1.38 MiB | 30.13 MiB/s, done.
Resolving deltas: 100% (245/245), done.


In [ ]:
!git clone https://github.com/jayeshsaita/Day-Night-Classifier

Cloning into 'Day-Night-Classifier'...
remote: Enumerating objects: 1303, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 1303 (delta 8), reused 27 (delta 5), pack-reused 1271
Receiving objects: 100% (1303/1303), 91.01 MiB | 33.18 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
!mkdir /content/CycleGAN-PyTorch/data/Day2night
!mkdir /content/CycleGAN-PyTorch/data/Day2night/train
!mkdir /content/CycleGAN-PyTorch/data/Day2night/train/A
!mkdir /content/CycleGAN-PyTorch/data/Day2night/train/B

In [ ]:
!cp /content/Day-Night-Classifier/day_night_dataset/train/day/* /content/CycleGAN-PyTorch/data/Day2night/train/A
!cp /content/Day-Night-Classifier/day_night_dataset/val/day/* /content/CycleGAN-PyTorch/data/Day2night/train/A
!cp /content/Day-Night-Classifier/day_night_dataset/train/night/* /content/CycleGAN-PyTorch/data/Day2night/train/B
!cp /content/Day-Night-Classifier/day_night_dataset/val/night/* /content/CycleGAN-PyTorch/data/Day2night/train/B

In [ ]:
!cp /content/a-PyTorch-Tutorial-to-Super-Resolution/models.py /content/CycleGAN-PyTorch/
!cp /content/a-PyTorch-Tutorial-to-Super-Resolution/utils.py /content/CycleGAN-PyTorch/

In [ ]:
!python3 /content/CycleGAN-PyTorch/train.py --dataroot /content/CycleGAN-PyTorch/data --dataset Day2night --cuda

In [ ]:
!wget https://drive.google.com/file/d/1EfNry1L-kX5cib7VO70mjHA7pYmtX995/

In [3]:
cd /content/a-PyTorch-Tutorial-to-Super-Resolution

/content/a-PyTorch-Tutorial-to-Super-Resolution


In [19]:
import argparse
import itertools
import os
import random

import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.utils as vutils
from PIL import Image
from tqdm import tqdm

srtransform = transforms.Compose([
                           transforms.Resize(int(256), Image.BICUBIC),
                           transforms.RandomCrop(256),
                          #  transforms.RandomHorizontalFlip(),
                           transforms.ToTensor()])

import time
import torch.backends.cudnn as cudnn
import torch
from torch import nn
from models import SRResNet
from datasets import SRDataset
from utils import *
import torch.nn.functional as F
# Data parameters
data_folder = './'  # folder with JSON data files
crop_size = 96  # crop size of target HR images
scaling_factor = 4  # the scaling factor for the generator; the input LR images will be downsampled from the target HR images by this factor

# Model parameters
large_kernel_size = 9  # kernel size of the first and last convolutions which transform the inputs and outputs
small_kernel_size = 3  # kernel size of all convolutions in-between, i.e. those in the residual and subpixel convolutional blocks
n_channels = 64  # number of channels in-between, i.e. the input and output channels for the residual and subpixel convolutional blocks
n_blocks = 16  # number of residual blocks

# Learning parameters
checkpoint = '/content/checkpoint_srresnet.pth.tar'  # path to model checkpoint, None if none
batch_size = 16  # batch size
start_epoch = 0  # start at this epoch
iterations = 1e6  # number of training iterations
workers = 4  # number of workers for loading data in the DataLoader
print_freq = 500  # print training status once every __ batches
lr = 1e-4  # learning rate
grad_clip = None  # clip if gradients are exploding

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cudnn.benchmark = True
checkpoint = torch.load(checkpoint)

model = checkpoint['model']
optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                                     lr=lr)
img = Image.open('/content/baboon.png', mode='r')
img = img.convert('RGB')
img =srtransform(img)
img = img.to(device)
imagenet_mean = torch.FloatTensor([0.485, 0.456, 0.406]).unsqueeze(1).unsqueeze(2)
imagenet_std = torch.FloatTensor([0.229, 0.224, 0.225]).unsqueeze(1).unsqueeze(2)
imagenet_mean_cuda = torch.FloatTensor([0.485, 0.456, 0.406]).to(device).unsqueeze(0).unsqueeze(2).unsqueeze(3)
imagenet_std_cuda = torch.FloatTensor([0.229, 0.224, 0.225]).to(device).unsqueeze(0).unsqueeze(2).unsqueeze(3)
img = img.unsqueeze(0)
print("min:",torch.min(img))
print("max:",torch.max(img))
if img.ndimension() == 3:
    img = (img - imagenet_mean) / imagenet_std
elif img.ndimension() == 4:
    img = (img - imagenet_mean_cuda) / imagenet_std_cuda



sr_imgs = model(img)
sr_imgs = (sr_imgs+1)/2
sr_imgs = F.interpolate(sr_imgs,size=512,mode='bilinear')
vutils.save_image(sr_imgs,f"/content/SR_translated.png", normalize=True)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warn

min: tensor(0., device='cuda:0')
max: tensor(0.9804, device='cuda:0')


In [ ]:
import torch

x = torch.randn((1, 3, 5,5))
print(x)


tensor([[[[ 1.7043,  0.1272,  2.2886,  0.5185,  0.5069],
          [-1.6438,  0.2764,  0.1816, -1.6745,  1.1881],
          [-0.4356,  0.0664, -1.6435, -0.9383, -2.2518],
          [-1.5022,  1.6561, -2.0535,  0.2920,  0.5376],
          [ 0.5800, -2.3558,  0.4539, -0.8913, -0.2394]],

         [[ 0.3562, -1.1237, -2.8899, -1.5435,  2.2437],
          [ 1.3330,  0.7521, -0.3104,  1.9953, -0.2463],
          [ 0.9327,  2.3944, -0.1064,  0.3872,  1.7508],
          [-0.7213,  0.2644,  1.2730,  0.1761, -0.9506],
          [-0.8950,  1.2898, -0.0570,  2.4474, -0.6436]],

         [[ 0.8123, -0.0111,  0.6140, -1.1497, -0.4176],
          [-1.6800, -1.1179, -0.0838, -1.1088,  0.2070],
          [ 0.2427, -0.6679,  0.8796,  1.6698, -1.2682],
          [-0.7439, -1.3174,  0.5307, -0.7725,  1.5377],
          [-1.3022, -2.4867, -0.1349, -0.8690,  0.3605]]]])


In [ ]:
x[:,[2,1,0],:]

tensor([[[[ 0.8123, -0.0111,  0.6140, -1.1497, -0.4176],
          [-1.6800, -1.1179, -0.0838, -1.1088,  0.2070],
          [ 0.2427, -0.6679,  0.8796,  1.6698, -1.2682],
          [-0.7439, -1.3174,  0.5307, -0.7725,  1.5377],
          [-1.3022, -2.4867, -0.1349, -0.8690,  0.3605]],

         [[ 0.3562, -1.1237, -2.8899, -1.5435,  2.2437],
          [ 1.3330,  0.7521, -0.3104,  1.9953, -0.2463],
          [ 0.9327,  2.3944, -0.1064,  0.3872,  1.7508],
          [-0.7213,  0.2644,  1.2730,  0.1761, -0.9506],
          [-0.8950,  1.2898, -0.0570,  2.4474, -0.6436]],

         [[ 1.7043,  0.1272,  2.2886,  0.5185,  0.5069],
          [-1.6438,  0.2764,  0.1816, -1.6745,  1.1881],
          [-0.4356,  0.0664, -1.6435, -0.9383, -2.2518],
          [-1.5022,  1.6561, -2.0535,  0.2920,  0.5376],
          [ 0.5800, -2.3558,  0.4539, -0.8913, -0.2394]]]])

In [ ]:
x+

TypeError: ignored